In [124]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
from gensim.models import Word2Vec
import multiprocessing
corpus=brown.sents()
import numpy as np
import random
import sys

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [0]:
emb_dim=300
wv= Word2Vec(corpus, size=emb_dim, sg=1, window=5, min_count=5, negative=15, iter=10, workers=multiprocessing.cpu_count())

In [126]:
f=wv.most_similar(positive=['car'], topn=5)
k=f[0][0]
#print(k)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [127]:
sen=wv.predict_output_word(k)
sent=str()
p=0
for i in sen:
  p=p+1
  if(p==6):
    sent=sent+k+' '
  sent=sent+i[0]+' '
sent

'then P T A pencil truck single Q on line at '

In [0]:
text=brown.words()
text=' '.join(text)
text=text.lower()


In [133]:
chars = sorted(list(set(text))) # getting all unique chars
print('total chars: ', len(chars))
print(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print(len(chars))

total chars:  58
[' ', '!', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}']
58


In [0]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [0]:
from tensorflow.keras.callbacks import LambdaCallback
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [0]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [139]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

Epoch 1/5
15956/15956 [==============================] - ETA: 0s - loss: 1.6408
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "10 , d is a diagonalizable operator whic"
10 , d is a diagonalizable operator which a state , and the reason and the complation of the problem . the programs , and the proficer . the complication . the state . the straight the provided to the state , and the propertion . the south the form of the state and state . the program . the state . the complication of the new count the proportional programs of the man . she said . `` the state , and the state . the manual man . the prog
----- diversity: 0.5
----- Generating with seed: "10 , d is a diagonalizable operator whic"
10 , d is a diagonalizable operator which had not a controlled the reason . he some will way . succeed and states black , and it is the longed the one is considerate not opened the same will be disting and a complation for the face , and the barscand . the b

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


 english containing the state and store and have been the state who came to extlitie , and in the parales , but the stuble and a companie the man . it is pallan . the original consideration . short of the police , and the coufsing to the sence of bare to the first at the gradul 
----- diversity: 1.0
----- Generating with seed: "ledge , ten miles north of the campus , "
ledge , ten miles north of the campus , shown mreshodiels in lack certain . . if he made out . as some up knet of a pulsharely , more on home of the striphed it a company and trues now a nagin concepts of the eyisnegs attentions , when copus af of writing ials when 3 1seasiny , most , felt '' . mollitt from sugner's intession of the fear about finds , with a still for the middle is day and tending , wherefor paying to be jephained aullo
----- diversity: 1.2
----- Generating with seed: "ledge , ten miles north of the campus , "
ledge , ten miles north of the campus , trel-equacuon . the far ariling sekrimers in orienan af

In [0]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    print(sentence)
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [122]:
print(generate_text(10, 0.2))

to aap use jarur de, aur aise react kare
to aap use jarur de, aur aise react kare ka karki 
